# I have chosen 5 ML Algorithms namely:


1. Naive Bayes
2. Stochastic Gradient Descent
3. Logistic Regression
4. Random Forest
5. Linear Suport Vector Machine

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm
import pickle

In [2]:
# Following is the list of flairs selected

flairs=["[R]eddiquette","Politics","Non-Political","Business/Finance","Unverified","Science/Technology","Sports","Photography","CAA-NRC","Food","AskIndia","Policy/Economy","Scheduled","AMA","Coronavirus"]

In [3]:
data = pd.read_csv('Stopwords.csv')
data.fillna('', inplace=True)

In [4]:
data.head(2)

,title,url,no_comm,body,comment,flair
0,"[""men"", ""thirty"", ""decided"", ""get"", ""married"",...",https://www.reddit.com/r/india/comments/fvy95j...,206,"[""corona"", ""virus"", ""given"", ""time"", ""think"", ...","[""plan"", ""finances"", ""work"", ""enjoy"", ""ways"", ...",AskIndia
1,"[""imfs"", ""nineteen"", ""growth"", ""prediction"", ""...",https://www.youtube.com/watch?v=QdGEw_JCZkc,4,,"[""karan"", ""thapar"", ""journalist"", ""made"", ""god...",Policy/Economy


#####  I have created a new column in my data 'combined'  which holds the values for columns combined from title, url , comment which gave me the highest accuracy levels, about 70% using the Logistic Regression Model

In [5]:
data['combined']=data['title']+","+data['url']+","+data['comment']

In [6]:
arr = data['combined']
print(arr)

0       ["men", "thirty", "decided", "get", "married",...
1       ["imfs", "nineteen", "growth", "prediction", "...
2       ["roasted", "chicken", "veggies"],https://i.re...
3       ["khan", "academy", "helped", "many", "us", "u...
4       ["world", "learn", "kerala", "fight", "covid19...
                              ...                        
1495    ["cop", "\u2019", "hand", "chopped", "two", "o...
1496    ["clashes", "erupt", "kashmir", "first", "civi...
1497    ["video", "mosque", "vandalised", "set", "fire...
1498    ["mobile", "internet", "speed", "slow", "india...
1499    ["twitter", "suspends", "kangana", "ranauts", ...
Name: combined, Length: 1500, dtype: object


In [7]:
arr = data['combined']
cnt=0
for ele in arr:
    a1=''
    a = str(ele).split(',')
    for item in a:
        item1 = item.replace("[",'')
        item1 = item1.replace("]",'')
        item1 = item1.replace('"','')
        item1 = item1.replace(' ','')
        a1=a1+","+(item1)
        data.at[cnt,'combined']=a1
    cnt=cnt+1
    

In [8]:
arr = data['combined']
print(arr)

0       ,men,thirty,decided,get,married,plan,old,age,h...
1       ,imfs,nineteen,growth,prediction,india,ludicro...
2       ,roasted,chicken,veggies,https://i.redd.it/eyt...
3       ,khan,academy,helped,many,us,understand,chapte...
4       ,world,learn,kerala,fight,covid19,https://www....
                              ...                        
1495    ,cop,\u2019,hand,chopped,two,others,injured,at...
1496    ,clashes,erupt,kashmir,first,civilian,death,co...
1497    ,video,mosque,vandalised,set,fire,ashok,nagar,...
1498    ,mobile,internet,speed,slow,india,pakistan,nep...
1499    ,twitter,suspends,kangana,ranauts,sister,rango...
Name: combined, Length: 1500, dtype: object


In [9]:
data.head()

,title,url,no_comm,body,comment,flair,combined
0,"[""men"", ""thirty"", ""decided"", ""get"", ""married"",...",https://www.reddit.com/r/india/comments/fvy95j...,206,"[""corona"", ""virus"", ""given"", ""time"", ""think"", ...","[""plan"", ""finances"", ""work"", ""enjoy"", ""ways"", ...",AskIndia,",men,thirty,decided,get,married,plan,old,age,h..."
1,"[""imfs"", ""nineteen"", ""growth"", ""prediction"", ""...",https://www.youtube.com/watch?v=QdGEw_JCZkc,4,,"[""karan"", ""thapar"", ""journalist"", ""made"", ""god...",Policy/Economy,",imfs,nineteen,growth,prediction,india,ludicro..."
2,"[""roasted"", ""chicken"", ""veggies""]",https://i.redd.it/eytftqoreda41.jpg,20,,"[""removed"", ""looks"", ""juicy"", ""nice"", ""could"",...",Food,",roasted,chicken,veggies,https://i.redd.it/eyt..."
3,"[""khan"", ""academy"", ""helped"", ""many"", ""us"", ""u...",https://www.reddit.com/r/india/comments/fqmz7y...,54,"[""indebted"", ""salmans"", ""efforts"", ""make"", ""bo...","[""good"", ""see"", ""people"", ""using"", ""khan"", ""ac...",AskIndia,",khan,academy,helped,many,us,understand,chapte..."
4,"[""world"", ""learn"", ""kerala"", ""fight"", ""covid19""]",https://www.technologyreview.com/2020/04/13/99...,0,,,Coronavirus,",world,learn,kerala,fight,covid19,https://www...."


In [10]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1500 entries, 0 to 1499
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   title     1500 non-null   object
 1   url       1500 non-null   object
 2   no_comm   1500 non-null   int64 
 3   body      1500 non-null   object
 4   comment   1500 non-null   object
 5   flair     1500 non-null   object
 6   combined  1500 non-null   object
dtypes: int64(1), object(6)
memory usage: 82.2+ KB


######  Here I have set up the training and testing variables for my algorithm. Finally I chose a 9:1 training and testing ratio as it provided me with the highest accuracy results.

I started off with a 8:2 split and the results were about 45% - 50% and then I started tweaking the split and finally settled for a 9:1 split.

In [11]:
#Dependent and independent variables
y = data.flair
X = data.combined

#Setting training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.1,random_state = 42)

#####  Using a Pipeline function combines the CountVetorizer command which cleans the data, the TfidfTransformer which computes word counts etc. and finally the Regression Model Command into single easy to understand command.

In [12]:
#Naive Bayes
nb = Pipeline([('vect', CountVectorizer()),
               ('tfidf', TfidfTransformer()),
               ('clf', MultinomialNB()),
              ])

NB = nb.fit(X_train, y_train)
y_pred = nb.predict(X_test)


print("Accuracy is "+str(accuracy_score(y_pred, y_test)))
print(classification_report(y_test, y_pred,target_names=flairs))


Accuracy is 0.5466666666666666
                    precision    recall  f1-score   support

     [R]eddiquette       0.24      1.00      0.39         7
          Politics       0.21      0.50      0.29        10
     Non-Political       1.00      0.33      0.50        12
  Business/Finance       0.38      0.83      0.53         6
        Unverified       1.00      0.58      0.74        12
Science/Technology       1.00      0.67      0.80         9
            Sports       0.00      0.00      0.00         9
       Photography       1.00      0.83      0.91        12
           CAA-NRC       0.20      0.40      0.27         5
              Food       0.67      0.62      0.64        13
          AskIndia       0.93      1.00      0.97        14
    Policy/Economy       1.00      0.14      0.25         7
         Scheduled       0.89      0.67      0.76        12
               AMA       1.00      0.09      0.17        11
       Coronavirus       0.67      0.36      0.47        11

       

In [13]:
#SGD
sgd = Pipeline([('vect', CountVectorizer()),
                ('tfidf', TfidfTransformer()),
                ('clf', SGDClassifier(loss='hinge', penalty='l2',alpha=1e-3, random_state=42, max_iter=5, tol=None)),
               ])
SGD = sgd.fit(X_train, y_train)
y_pred = sgd.predict(X_test)

print("Accuracy is "+str(accuracy_score(y_pred, y_test)))
print(classification_report(y_test, y_pred,target_names=flairs))

Accuracy is 0.64
                    precision    recall  f1-score   support

     [R]eddiquette       0.70      1.00      0.82         7
          Politics       0.38      0.50      0.43        10
     Non-Political       0.75      0.50      0.60        12
  Business/Finance       0.42      0.83      0.56         6
        Unverified       0.77      0.83      0.80        12
Science/Technology       0.56      0.56      0.56         9
            Sports       0.67      0.22      0.33         9
       Photography       0.63      1.00      0.77        12
           CAA-NRC       0.43      0.60      0.50         5
              Food       0.62      0.62      0.62        13
          AskIndia       0.88      1.00      0.93        14
    Policy/Economy       0.75      0.43      0.55         7
         Scheduled       0.92      0.92      0.92        12
               AMA       0.33      0.09      0.14        11
       Coronavirus       0.50      0.36      0.42        11

          accuracy   

######  As the Logistic Regression Model had the largest accuracy for the 9:1 split, I selected the following model and the model for saved using the pickle function.

In [12]:
#Logistic Regression
logreg = Pipeline([('vect', CountVectorizer()),
                ('tfidf', TfidfTransformer()),
                ('clf', LogisticRegression(n_jobs=1, C=1e20)),
               ])
LOGREG = logreg.fit(X_train, y_train)
y_pred = logreg.predict(X_test)
pickle.dump(LOGREG,open('model_final.pkl','wb'))
print("Accuracy is "+str(accuracy_score(y_pred, y_test)))
print(classification_report(y_test, y_pred,target_names=flairs))

Accuracy is 0.6933333333333334
                    precision    recall  f1-score   support

     [R]eddiquette       1.00      1.00      1.00         7
          Politics       0.29      0.40      0.33        10
     Non-Political       0.86      0.50      0.63        12
  Business/Finance       0.71      0.83      0.77         6
        Unverified       0.91      0.83      0.87        12
Science/Technology       0.75      0.67      0.71         9
            Sports       0.57      0.44      0.50         9
       Photography       0.85      0.92      0.88        12
           CAA-NRC       0.40      0.80      0.53         5
              Food       0.67      0.62      0.64        13
          AskIndia       0.93      0.93      0.93        14
    Policy/Economy       0.75      0.86      0.80         7
         Scheduled       0.92      0.92      0.92        12
               AMA       0.50      0.45      0.48        11
       Coronavirus       0.40      0.36      0.38        11

       

In [15]:
#Random Forest
ranfor = Pipeline([('vect', CountVectorizer()),
                   ('tfidf', TfidfTransformer()),
                   ('clf', RandomForestClassifier(n_estimators = 1000, random_state = 5)),
                  ])
RM = ranfor.fit(X_train, y_train)
#pickle.dump(RM,open("model_RM.sav",'wb'))
y_pred = ranfor.predict(X_test)

print("Accuracy is "+str(accuracy_score(y_pred, y_test)))
print(classification_report(y_test, y_pred,target_names=flairs))

Accuracy is 0.5933333333333334
                    precision    recall  f1-score   support

     [R]eddiquette       0.64      1.00      0.78         7
          Politics       0.33      0.50      0.40        10
     Non-Political       0.56      0.42      0.48        12
  Business/Finance       0.36      0.83      0.50         6
        Unverified       0.60      0.75      0.67        12
Science/Technology       0.50      0.22      0.31         9
            Sports       0.60      0.33      0.43         9
       Photography       0.53      0.83      0.65        12
           CAA-NRC       0.33      0.40      0.36         5
              Food       0.70      0.54      0.61        13
          AskIndia       0.93      1.00      0.97        14
    Policy/Economy       0.56      0.71      0.63         7
         Scheduled       0.86      1.00      0.92        12
               AMA       0.50      0.09      0.15        11
       Coronavirus       1.00      0.18      0.31        11

       

In [16]:
#SVM
svm1 = Pipeline([('vect', CountVectorizer()),
                ('tfidf', TfidfTransformer()),
                ('clf', svm.SVC(kernel='linear', C=1.0)),
               ])
SVM = svm1.fit(X_train, y_train)
y_pred = svm1.predict(X_test)

print("Accuracy is "+str(accuracy_score(y_pred, y_test)))
print(classification_report(y_test, y_pred,target_names=flairs))

Accuracy is 0.6866666666666666
                    precision    recall  f1-score   support

     [R]eddiquette       1.00      1.00      1.00         7
          Politics       0.29      0.40      0.33        10
     Non-Political       0.75      0.50      0.60        12
  Business/Finance       0.62      0.83      0.71         6
        Unverified       0.77      0.83      0.80        12
Science/Technology       0.75      0.67      0.71         9
            Sports       0.71      0.56      0.63         9
       Photography       0.79      0.92      0.85        12
           CAA-NRC       0.29      0.40      0.33         5
              Food       0.69      0.69      0.69        13
          AskIndia       0.93      1.00      0.97        14
    Policy/Economy       0.46      0.86      0.60         7
         Scheduled       1.00      0.83      0.91        12
               AMA       0.57      0.36      0.44        11
       Coronavirus       0.67      0.36      0.47        11

       